## 3-1

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://news.naver.com/"

ages = (10, 20, 30, 40, 50, 60)
news = {'연령': [], '제목': [], '기사 내용 요약본': [], '신문사': [], '조회수': [], '원문 url': []}

html = requests.get(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})
soup = BeautifulSoup(html.text, 'html.parser')

In [2]:
presslist = soup.select("#_rankingList0 .list_press")
titlelist = soup.select("#_rankingList0 .list_tit")

tmp = []
for i in range(len(titlelist)):
    tmp.append([presslist[i].text, titlelist[i].text])

In [3]:
# 3-1
print(tmp)

[['연합뉴스TV', '입양 2시간 만에 진돗개 모녀 도살…70대 구속'], ['중앙일보', "라이터 갖다 대자 불 '화르륵'…중국 한 마을의 이상한 수 …"], ['조선일보', '미친 전세에 지친 무주택자, 노·도·강부터 샀다'], ['연합뉴스', '노량진발 확진자 1명 교원 임용시험 응시…형평성 논란 불가 …'], ['시사IN', '‘균형 잡힌’ 방역이라야 지속가능하다']]


## 3-2

In [4]:
url = "https://news.naver.com/main/officeList.nhn"
# 크롤링 차단하고 있으므로 User-Agent 지정하여 우회
html = requests.get(url, headers={"User-Agent" : "Mozilla/5.0"})
soup = BeautifulSoup(html.text, 'html.parser')

# 신문 제공사 딕셔너리
# 신문사이름: 신문사번호
newspapers = {}

# css selector 이용하여 파싱
for ele in soup.select('ul.group_list a'):
    newspapers[ele.text] = (ele['href'].split('&oid=')[1])

# 파라미터 붙어서 수동할당
newspapers['연합뉴스'] = '001'

print(newspapers)

{'경향신문': '032', '국민일보': '005', '동아일보': '020', '문화일보': '021', '서울신문': '081', '세계일보': '022', '조선일보': '023', '중앙일보': '025', '한겨레': '028', '한국일보': '469', '뉴스1': '421', '뉴시스': '003', '연합뉴스': '001', '연합뉴스TV': '422', '채널A': '449', '한국경제TV': '215', 'JTBC': '437', 'KBS': '056', 'MBC': '214', 'MBN': '057', 'SBS': '055', 'SBS CNBC': '374', 'TV조선': '448', 'YTN': '052', '매일경제': '009', '머니투데이': '008', '서울경제': '011', '아시아경제': '277', '이데일리': '018', '조선비즈': '366', '조세일보': '123', '파이낸셜뉴스': '014', '한국경제': '015', '헤럴드경제': '016', '노컷뉴스': '079', '더팩트': '629', '데일리안': '119', '머니S': '417', '미디어오늘': '006', '아이뉴스24': '031', '오마이뉴스': '047', '프레시안': '002', '디지털데일리': '138', '디지털타임스': '029', '블로터': '293', '전자신문': '030', 'ZDNet Korea': '092', '레이디경향': '145', '매경이코노미': '024', '시사IN': '308', '시사저널': '586', '신동아': '262', '월간 산': '094', '이코노미스트': '243', '주간경향': '033', '주간동아': '037', '주간조선': '053', '중앙SUNDAY': '353', '한겨레21': '036', '한경비즈니스': '050', '기자협회보': '127', '뉴스타파': '607', '동아사이언스': '584', '여성신문': '310', '일다': '00

In [5]:
from datetime import datetime

In [6]:
# 위 출력된 리스트 중 하나 산택
# company = '동아일보'

news = {'신문사': [], '제목': [], 'link': []}

def startparse(company):
    # 오늘날짜 선택
    date = datetime.today().strftime("%Y%m%d")

    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학, 오피니언 중 선택
    # subject = '정치'

    # 분류 딕셔너리
    sids = {'정치': '100', '경제': '101', '사회': '102', '생활/문화': '103', '세계': '104', 'IT/과학': '105', '오피니언': '110'}

    # 신문사 넘버
    company_num = newspapers[company]

    # 분야 번호
    # sid = sids[subject]

    # dataFrame dictionary
    # news = {'신문사': [], '제목': [], 'link': []}

    for sid in sids:
        # 뉴스 페이지들 끝 번호 알아내기
        url = "https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1={}&mid=sec&oid={}&date={}&page=9999".format(sids[sid], company_num, date)
        html = requests.get(url, headers={"User-Agent" : "Mozilla/5.0"})
        soup = BeautifulSoup(html.text, 'html.parser')
        pages = soup.select('div.paging strong')

        lastpage = int(pages[-1].text)

        # 각 페이지마다
        for page in range(lastpage):
            list_url = "https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1={}&mid=sec&oid={}&date={}&page={}".format(sids[sid], company_num, date, page)
            list_html = requests.get(list_url, headers={"User-Agent" : "Mozilla/5.0"})
            list_soup = BeautifulSoup(list_html.text, 'html.parser')

            print(list_url)

            # 뉴스 인덱스
            elem_index = 0
            # 각 뉴스 기사마다
            for element in list_soup.select('div.list_body>ul a'):
                # URL에 날짜를 줘도 다른 날짜의 기사도 같이 출력됨을 확인
                # 따라서 수동으로 날짜 다를 경우 넘어가게 조치
                article_date = list_soup.select('span.date')[elem_index].text
                #if pd.to_datetime(article_date.split()[0]) != pd.to_datetime(date): elem_index += 1; continue

                link = element['href']
                title = element.text

                for ele in titlelist:
                    if ele.text[:10] in title:
                        print(title, article_date)
                        news['신문사'].append(company)
                        news['제목'].append(title)
                        news['link'].append(link)
                        return news

                elem_index += 1;

In [7]:
for ele in tmp:
    startparse(ele[0])

dataframe = pd.DataFrame(news)
dataframe

https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=100&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=101&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=102&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=103&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=104&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=105&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=110&mid=sec&oid=422&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=100&mid=sec&oid=025&date=20201124&page=0
https://news.naver.com/main/list.nhn?mode=LSD&listType=title&sid1=101&mid=sec&oid=025&date=20201124&page=0
https://news.naver.com/main/list.nhn?

,신문사,제목,link


In [8]:
# 저장
dataframe = pd.DataFrame(news)
dataframe.to_csv('news.csv', encoding='utf-8-sig', index=False)